

### 🎓 LearnMate: A Personalized AI Learning Coach

LearnMate is a Retrieval-Augmented Generation (RAG) system designed to act as an AI learning coach. It provides personalized learning pathways and course recommendations based on a user's interests, skill level, and career goals.

The project is built as a self-contained Python notebook, making it easy to share and run.

-----

### 💡 Proposed Solution

LearnMate's core function is to guide students by:

  * **Providing Personalized Pathways**: It suggests specific courses and learning roadmaps in domains like Frontend Development, Cybersecurity, and UI/UX.
  * **Offering Contextual Advice**: It explains why a particular course fits a user's goals and provides general tips for learning and progression.
  * **Creating an Interactive Interface**: The project uses Gradio to create a user-friendly web interface, allowing for a conversational experience.

-----

### ⚙️ Tools & Technologies

This project is built using the following key technologies:

  * **LangChain**: The primary framework for building the RAG pipeline.
  * **Replicate**: Used to access and run the `ibm-granite/granite-3.3-8b-instruct` large language model.
  * **Gradio**: Used to create a simple and interactive web interface for the application.
  * **FAISS**: An efficient library for vector search, used here to create a vector store for the knowledge base.
  * **HuggingFace Embeddings**: A model from Hugging Face used to generate the vector embeddings for the text data.
  * **Python-dotenv**: A crucial library for securely loading API keys from a `.env` file.

-----

### 🚀 Getting Started

Follow these steps to set up and run the LearnMate project on your local machine.

#### Prerequisites

  * Python 3.8+
  * A Replicate API Token

#### Installation

1.  **Clone the Repository**:
    ```bash
    git clone https://github.com/your-username/your-repo-name.git
    cd your-repo-name
    ```
2.  **Create a `.env` file**:
    Create a file named `.env` in the root directory and add your Replicate API key:
    ```
    REPLICATE_API_TOKEN="your_token_here"
    ```
3.  **Install Dependencies**:
    The required libraries are listed in `requirements.txt`. Install them by running:
    ```bash
    pip install -r requirements.txt
    ```

#### Running the Application

Open the `LearnMate.ipynb` file in a Jupyter environment (like Jupyter Notebook, JupyterLab, or VS Code) and run all the cells in order. This will launch the Gradio interface, and a shareable link will be provided in the output.

-----

### 📄 How It Works

1.  **Dependencies & Authentication**: The notebook first installs all necessary libraries and then securely loads the Replicate API token from the `.env` file.
2.  **RAG Pipeline**:
      * **Data Loading**: A small "knowledge base" of courses is created in a text file (`learnmate_courses.txt`).
      * **Text Splitting**: The text is split into smaller chunks to prepare it for embedding.
      * **Embedding & Vector Store**: `HuggingFaceEmbeddings` is used to convert the text chunks into vectors, which are then stored in a `FAISS` vector database for efficient retrieval.
3.  **LLM & RAG Chain**:
      * A connection to the **IBM Granite 3.3B Instruct** model on Replicate is established.
      * A **Retrieval-Augmented Generation (RAG)** chain is constructed. This chain retrieves the most relevant information from the vector store based on the user's query and then uses the large language model to generate a natural, helpful response.
4.  **User Interface**: The `gradio` library creates a simple web interface that allows a user to type a question and receive a response from the AI coach.

In [ ]:

!pip install replicate langchain gradio langchain_community langchain_huggingface faiss-cpu transformers python-dotenv


import os
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

# Check if the API token is loaded
replicate_api_token = os.getenv("REPLICATE_API_TOKEN")
if not replicate_api_token:
    raise ValueError("REPLICATE_API_TOKEN not found. Please set it in your .env file or Colab Secrets.")

# ==============================================================================
# ==============================================================================
import gradio as gr
from langchain_community.vectorstores import FAISS
from langchain_community.llms import Replicate
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from transformers import AutoTokenizer
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA, LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain_community.embeddings import HuggingFaceEmbeddings

# IBM Granite model via Replicate
model_path = "ibm-granite/granite-3.3-8b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = Replicate(
    model=model_path, 
    replicate_api_token=replicate_api_token, # Use the securely loaded token
    input={"temperature": 0.1, "max_length": 500}
)

# 📚 Sample LearnMate knowledge base
filename = "learnmate_courses.txt"
with open(filename, "w") as f:
    f.write("""
Frontend Development Pathway:
1. HTML & CSS Fundamentals (Beginner) — Learn basic web page structure and styling.
2. JavaScript Essentials (Beginner) — Add interactivity to websites.
3. React for Developers (Intermediate) — Build dynamic, modern web apps.
4. Frontend Capstone Project (Advanced) — Create a real-world portfolio project.

Cybersecurity Pathway:
1. Intro to Cybersecurity (Beginner) — Understand basic security concepts.
2. Network Security (Intermediate) — Learn to secure communication channels.
3. Ethical Hacking (Advanced) — Test systems for vulnerabilities ethically.

UI/UX Design Pathway:
1. Intro to UI/UX (Beginner) — Learn the basics of user interface and experience.
2. Wireframing & Prototyping (Intermediate) — Create interactive prototypes.
3. UI/UX Capstone (Advanced) — Complete a design project for a real client.

Tips for Learning:
- Always build projects alongside learning theory.
- Review and adapt your learning roadmap every 2 months.
- Start from your current skill level and progress gradually.
""")

# 📄 Load & Split content
loader = TextLoader(filename)
documents = loader.load()
splitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=50)
texts = splitter.split_documents(documents)

# 🔍 Vector store (FAISS for quick Colab testing)
# The `HuggingFaceEmbeddings` import is correct and will use the model locally
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vector_db = FAISS.from_documents(texts, embeddings)

# 🎯 Prompt for LearnMate
template = """
You are LearnMate, an AI learning coach.
User Question: {question}
Based on the student's interests, skill level, and goals, give a personalized learning pathway.
Suggest specific courses, explain why they fit, and provide tips for progression.
"""
prompt = PromptTemplate(template=template, input_variables=["question"])

# 🔗 Chains
llm_chain = LLMChain(llm=model, prompt=prompt)
combine_chain = StuffDocumentsChain(llm_chain=llm_chain)

rag_chain = RetrievalQA(
    retriever=vector_db.as_retriever(),
    combine_documents_chain=combine_chain,
    return_source_documents=False
)

# ==============================================================================
# ==============================================================================
def ask_learnmate(query):
    """
    Function to handle the Gradio input and generate a response.
    """
    if not query:
        return "Please enter a question."

    try:
        response = rag_chain.run(query)
        return response
    except Exception as e:
        return f"❌ An error occurred: {str(e)}"

iface = gr.Interface(
    fn=ask_learnmate,
    inputs=gr.Textbox(label="Ask LearnMate", placeholder="e.g. How do I become a frontend developer?"),
    outputs=gr.Textbox(label="LearnMate's Advice"),
    title="LearnMate — Personalized Course Pathways",
    description="Get a personalized learning roadmap for your career goals in frontend development, cybersecurity, or UI/UX.",
    theme="default"
)

# Launch the interface
iface.launch(share=True)